In [1]:
import sys
sys.path.append('..')
from config_notebooks import *
from qdrant.qdrant_helper import *
from sentence_transformers import CrossEncoder

c:\Ambarish\LEGAL\text2sql-legal\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
search = QdrantHelper(QDRANT_URL,
                      QDRANT_KEY,
                      MODEL_NAME,
                      QDRANT_COLLECTION,
                      RESULTS_LIMIT = 25,
                      CROSS_ENCODER_USE=True,)

In [3]:
query = """
What is Naked Truth ?
"""

In [4]:
results_to_return, \
metadata_source_filename_to_return, \
metadata_source_page_to_return, \
reranker_score_to_return = search.get_search_results(user_input=query,CATEGORY="")


What is Naked Truth ?

################################
category = 
user_id = None
Results to return Length: 25


In [7]:
metadata_source_filename_to_return

['legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'special_marriage_act.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'special_marriage_act.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'special_marriage_act.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'legalguide.pdf',
 'special_marriage_act.pdf']

In [8]:
#We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

cross_score_list =[]

for hit in results_to_return:
    cross_score = cross_encoder.predict([query,hit])
    cross_score_list.append(cross_score)

# Combine all lists into a single list of tuples
combined_lists = list(zip(results_to_return, \
metadata_source_filename_to_return, \
metadata_source_page_to_return, \
cross_score_list))

# Sort combined lists based on the float values in the last position (index 3)
sorted_combined = sorted(combined_lists, key=lambda x: x[3], reverse=True)

# Unpack the sorted combined list back into separate lists
results_to_return_sorted, \
metadata_source_filename_to_return_sorted, \
metadata_source_page_to_return_sorted, \
cross_score_list_sorted= zip(*sorted_combined)

# Convert tuples back to lists
results_to_return_sorted = list(results_to_return_sorted)
metadata_source_filename_to_return_sorted = list(metadata_source_filename_to_return_sorted)
metadata_source_page_to_return_sorted = list(metadata_source_page_to_return_sorted)
cross_score_list_sorted = list(cross_score_list_sorted)

In [11]:
print("Results after re-ranking")
print("Cross-encoder score: ", cross_score_list_sorted)

Results after re-ranking
Cross-encoder score:  [-4.5817213, -7.764212, -8.724777, -9.041127, -9.938349, -9.988352, -10.066312, -10.133227, -10.188252, -10.231129, -10.446102, -10.462021, -10.483532, -10.52917, -10.594371, -10.636823, -10.638062, -10.644715, -10.760315, -10.827051, -11.012038, -11.058759, -11.060701, -11.080072, -11.186394]
